# Challenge - Clinical Text Classification

#### You now know the structure of a Machine Learning pipeline, and how to implement it from the pre-processing of data to the evaluation of results. Good job! But can you do it with text data? In this challenge, you will have the opportunity to implement text processing techniques, including a simple strategy to represent text data in a structure that is compatible with Machine Learning algorithms: the Bag of Words model. Leveraging this strategy, and combining it with what you have learned by implementing the previous pipeline, you will be able to classify clinical notes according to the medical specialty where they were produced. Good luck!

Import Packages

In [ ]:
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

!pip install scikit-learn
!pip install nltk
!pip install pandas

In [ ]:
import pandas as pd
from sklearn import feature_extraction
from sklearn.metrics import balanced_accuracy_score, f1_score
from collections import Counter
from random import randint
import nltk
nltk.download('punkt')

from sklearn.model_selection import StratifiedShuffleSplit

## Load Data

In [ ]:
data = pd.read_csv("nlp_challenge.csv")

How does the data look? Here are a few examples:

In [ ]:
data.head()

The "description" columns contains the samples of the data you will use to train your model. The target is the "medical_specialty" column. As you can see, these data is very different from time series! Start by isolating the information you need (i.e., create the variables X_data and y_data, referring to the description and medical_specialty columns, respectively).

In [ ]:
X_data, y_data = data["description"], data["medical_specialty"]

## Split Data

Now, notice that your data is not split into train and test sets. Splitting data is one of the most defining steps in the Machine Learning pipeline. A bad split could result in data leakage, or a test set that does not accurately represent the true data distribution, which could consequently lead to over or under-estimation of results.

To avoid falling into this pits, one must know the data one's working with. Let us check the class distributions. To do so, we will make use of the Counter from the collections library, which gives us the occurrences of each unique instance within a list.

In [ ]:
Counter(y_data)

We want to make sure this distribution is similar both in training and testing. Apply a stratified method to do the split (https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html)

In [ ]:
train_idxs, test_idxs = next(StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=32).split(X_data, y_data))

In [ ]:
X_train, y_train, X_test, y_test = X_data[train_idxs], y_data[train_idxs], X_data[test_idxs], y_data[test_idxs]

Verify the class distributions on both resultant sets

In [ ]:
Counter(y_train) # Add code to obtain the result in terms of percentage

In [ ]:
Counter(y_test) # Add code to obtain the result in terms of percentage

## Text Preprocessing

Let's get some words rolling! Here's a tutorial of text processing techniques: https://towardsdatascience.com/text-preprocessing-in-natural-language-processing-using-python-6113ff5decd8
You should complement the function "preprocess_text", to return a list of clean tokens, when receiving a chunk of text. Note that the output of this function must be tokenized, to be compatible with the following steps of the notebook

In [ ]:
def preprocess_text(text):
    """Takes a text and returns the processed version of it.

    Args:
        text (str): raw text

    Returns:
        list: set of clean tokens containing the content of text
    """

    # Normalize Text (lowercasing)
    processed_text = text.lower()

    # Add more preprocessing steps below:

    # Try a few preprocessing techniques:
    # - Tokenization                       <---- Essential
    # - Stopwords Removal
    # - Stemming/Lemmatization

    # You can also implement other preprocessing you may find useful

    return processed_text

Test the function on a randomly sampled tweet from the train dataset (Give it a couple of tries to really see impact).

In [ ]:
random_description = X_train.values[randint(0, X_train.shape[0])]
print(f"Original Note: {random_description}")
print(f"Processed Note: {preprocess_text(random_description)}")

Now that you have cleaned every entry in our dataset, you can proceed to extract features. There are many ways to do this, but we will focus on two simple techniques: The bag-of-words model, and the tfidf. In the following cells, you'll be able to try several these two ways of vectorizing the data.

## Feature Extraction

Machine Learning models do not accept strings as input. So how can we train one to perform text classification? We need to represent each text sample into a numeric vector. This numeric vector is our feature vector. In this challenge, you will implement two simple methods for text vectorization: The bag-of-words and the tfidf methodologies. Here's a quick introduction to both of them: https://machinelearningmastery.com/gentle-introduction-bag-words-model/. These functions are already implemented. However, you must learn the processes behind these implementations, according to this article. So read it carefully.

### Bag-of-Words

To implement a bag-of-words vectorization, we will use the CountVectorizer function from sklearn (https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)

###### Note: Notice that this function expects the output from the preprocessing function to be a tokenized tweet. If you did not implement a tokenizer yet, you must re-think your preprocessing methodology.

In [ ]:
def get_bow_representations(train_samples, test_samples, tokenizer):
    """Returns a bag-of-words based representation of both the train and test samples.

    Args:
        train_samples (list): List of training samples.
        test_samples (list): List of test samples.
        tokenizer (object): A preprocessing function that outputs a list of tokens.

    Returns:
        train_vectors, test_vectors: vectorized representations of the train and test sets, according to the BOW method.
    """

    count_vectorizer = feature_extraction.text.CountVectorizer(tokenizer=tokenizer)

    train_vectors = count_vectorizer.fit_transform(train_samples) # YOU FIT TO TRAIN DATA

    test_vectors = count_vectorizer.transform(test_samples) # AND APPLY IT TO BOTH TEST AND TRAIN DATA

    return train_vectors, test_vectors

Test this function and check the dimension of the resultant vectors:

In [ ]:
train_vectors, test_vectors = get_bow_representations(X_train, X_test, preprocess_text)
print(train_vectors[0].shape)

#### Considering the result of the previous cell, what is the number of unique words in the entire preprocessed train dataset?

Answer here: 

### TF-IDF

TF-IDF implementation is similar to the Bag-of-Words one. But instead, we use the TfidfVectorizer (https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

In [ ]:
def get_tfidf_representations(train_samples, test_samples, tokenizer):
    """Returns a tf-idf based representation of both the train and test samples.

    Args:
        train_samples (list): List of training samples.
        test_samples (list): List of test samples.
        tokenizer (object): A preprocessing function that outputs a list of tokens.

    Returns:
        train_vectors, test_vectors: vectorized representations of the train and test sets, according to the BOW method.
    """

    tfidf_vectorizer = feature_extraction.text.TfidfVectorizer(tokenizer=tokenizer)

    train_vectors = tfidf_vectorizer.fit_transform(train_samples)

    test_vectors = tfidf_vectorizer.transform(test_samples)

    return train_vectors, test_vectors

Test this function, similarly to what you did earlier

In [ ]:
train_vectors, test_vectors = get_tfidf_representations(X_train, X_test, preprocess_text)
train_vectors

## Models

Now, you need a model to perform the main task: Text Classification of Clinical Descriptions. As so, we give you predictive functions that accept a model as input (You can choose your own classifier), train it upon training samples, and return a prediction against the test set.

#### Training Implementation

In [ ]:
def get_predictions(model, train_samples, train_labels, test_samples):
    """Simple implementation of a Naive Bayes classifier.

    Args:
        train_samples (_type_): List of vectorized trained tweets.
        train_labels (_type_): List of train labels.
        test_samples (_type_): List of vectorized test tweets.

    Returns:
        preds: Predictions against the test set.
    """

    model.fit(train_samples, train_labels)

    return model.predict(test_samples)

## Build your Pipeline!

Time to bring it all together! You got your preprocessing function, your feature extractors, and your training implentation. Now, you can combine them according to the structure of the traditional NLP pipeline you learned about in this first class.

1. Start by getting your data in the correct format, i.e. a set of training tweets, a set of training labels, a set of test tweets, and a set of test labels.

In [ ]:
# This step was already performed when we splitted the data!

2. Preprocess and get a numerical representation of the tweets. You should perform these two steps at once, since the vectorizers accept a preprocessing function as input.

In [ ]:
# Vectorize your text
# Notice that you don't have to apply pre-processing first, because the vetorizers apply it themselves.
# You must pass our preprocessing function to the feature extraction function, though.
x_train_vectorized, x_test_vectorized = # INSERT YOUR CODE HERE

3. Get predictions against the test set, using a classifier of your choice

In [ ]:
# Choose your fighter
model = # INSERT YOUR CODE HERE

In [ ]:
# Train your model and get predictions against the test samples
preds = # INSERT YOUR CODE HERE

4. Check the performance you achieve through the selected pipeline.

In [ ]:
# Test your model in terms of accuracy, f1-score
# REPLACE THE None's FOR THE RIGHT VARIABLES

print(f"Accuracy: {round(balanced_accuracy_score(None, None), 5)}")
print(f"F1-score: {round(f1_score(None, None, average='macro'), 5)}")

If you managed to reach this cell without any issues, you probably saw that you can reach a quite reasonable results with this simple pipeline (for a classical 4-class problem, a random classifier would be right 25% of the times). Congratulations, you just built a decent text classifier capable of attributing a description to a certain medical specialty!

#### Challenge: Note that this is a very simple task, since clinical note descriptions are already a summary of the most relevant traits of each clinical note! To achieve these results with the notes themselves would be much harder! Try to reproduce the pipeline you created, but use the column "transcription" of the original dataset as the X_data, instead.